# Araç Tespiti (YOLO) - Benim Öğrenme Yolculuğum

Bu defterde, araç tespitini adım adım kendi gözümden kuruyorum. Amacım; veriyi doğru okumak, YOLO'nun istediği formata dönüştürmek, küçük bir keşif analizi yapmak ve en sonunda eğitim için hazır bir klasör yapısı ve ayar dosyası üretmek.

- Önce veriyi okuyup kutuları YOLO formatına çeviriyorum (x_center, y_center, w, h).
- Sonra veriyi eğitim ve doğrulama olarak ikiye ayırıp `yolo_dataset/` yapısını kuruyorum.
- Birkaç görseli ekrana getirip “gerçekten doğru mu yapıyorum?” diye kendimi kontrol ediyorum.
- Eğitimde kullanabileceğim basit bir `yolo_config.yaml` hazırlıyorum.

Tek sınıflı (car) bir tespit problemi üzerinde çalışıyorum; yani modelim sadece araçları tanımaya odaklanacak.


## Kurulum ve Kütüphaneler
Burada ihtiyacım olan paketleri içe aktarıyorum. Kaggle’da bazen bazı paketler hazır geliyor, bazen gelmiyor; o yüzden `pip install` satırını yoruma aldım, gerekirse açıp çalıştırırım. Amacım, defterin başında ortamı sağlama almak.


In [ ]:
# !pip -q install opencv-python-headless tqdm matplotlib pandas numpy torch

import os, time, random
import numpy as np
import pandas as pd
import cv2
import torch
from tqdm.auto import tqdm
import shutil as sh

from IPython.display import Image, clear_output, display
import matplotlib.pyplot as plt

img_h, img_w, num_channels = (380, 676, 3)


## Veriyi Okuyorum ve YOLO’ya Hazırlıyorum
Bu adımda elimdeki `train_solution_bounding_boxes.csv` dosyasını okuyorum. Kutular piksel cinsinden geldiği için önce merkez (x_center, y_center) ve boyut (w, h) hesaplıyorum; sonra YOLO’nun istediği gibi görüntü boyutuna bölerek normalize ediyorum. Böylece model, farklı çözünürlüklerde de aynı mantıkla kutuları anlayabiliyor.


In [ ]:
df = pd.read_csv('train_solution_bounding_boxes.csv')
df.rename(columns={'image':'image_id'}, inplace=True)
df['image_id'] = df['image_id'].apply(lambda x: x.split('.')[0])
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['classes'] = 0

df['x_center'] = df['x_center']/img_w
df['w'] = df['w']/img_w
df['y_center'] = df['y_center']/img_h
df['h'] = df['h']/img_h

display(df.head())

# benzersiz image_id listesi
index = list(set(df.image_id))
print('Toplam görüntü:', len(index))


## Küçük Bir Gerçeklik Kontrolü
Burada rastgele bir görüntü seçip gerçekten doğru klasörden okuyabiliyor muyum diye bakıyorum. Boyutlar ve dosya yolları doğruysa, sonraki adımlarda hata almam daha düşük bir ihtimal oluyor.


In [ ]:
image = random.choice(index)
print(f"Image ID: {image}")
img_path = f'training_images/{image}.jpg'
print(f"Görüntü yolu: {img_path}")

img = cv2.imread(img_path)
if img is not None:
    print(f"Görüntü boyutları: {img.shape}")
    print("Görüntü başarıyla yüklendi!")
else:
    print("Görüntü yüklenemedi, alternatif yollar deneniyor...")
    alternative_paths = [
        f'training_images/{image}.jpg',
        f'./training_images/{image}.jpg',
        f'../training_images/{image}.jpg'
    ]
    for path in alternative_paths:
        if os.path.exists(path):
            print('Bulundu:', path)
            img = cv2.imread(path)
            if img is not None:
                print(f"Görüntü boyutları: {img.shape}")
                break
        else:
            print('Dosya bulunamadı:', path)

if img is None:
    print('Hiçbir yolda görüntü bulunamadı!')
    if os.path.exists('training_images'):
        files = os.listdir('training_images')[:5]
        print('training_images içinden ilk 5 dosya:', files)
    else:
        print('training_images klasörü bulunamadı!')

if img is not None:
    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f'Rastgele Görsel: {image}')
    plt.axis('off')


## YOLO Veri Kümesi Oluşturma (Mutfağın Kalbi)
Şimdi en kritik kısma geldim: veriyi YOLO’nun anlayacağı düzene sokmak. Hem `labels/` hem de `images/` klasörlerini oluşturup, veriyi eğitim ve doğrulama olarak ayırıyorum. Her görüntü için `.txt` dosyası oluşturup, kutu bilgilerini satır satır yazıyorum.


In [ ]:
def create_yolo_dataset(df, index, source_dir='training_images', output_dir='yolo_dataset'):
    print('YOLO veri seti oluşturuluyor...')
    print('Kaynak klasör:', source_dir)
    print('Çıktı klasörü:', output_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print('Çıktı klasörü oluşturuldu:', output_dir)

    val_ratio = 0.2
    val_size = int(len(index) * val_ratio)
    val_index = random.sample(index, val_size)

    print('Toplam görüntü:', len(index))
    print('Eğitim görüntüsü:', len(index) - val_size)
    print('Doğrulama görüntüsü:', val_size)

    # Klasörler
    train_labels_dir = os.path.join(output_dir, 'labels', 'train')
    val_labels_dir = os.path.join(output_dir, 'labels', 'val')
    train_images_dir = os.path.join(output_dir, 'images', 'train')
    val_images_dir = os.path.join(output_dir, 'images', 'val')
    for d in [train_labels_dir, val_labels_dir, train_images_dir, val_images_dir]:
        os.makedirs(d, exist_ok=True)
    print('Klasör yapısı hazırlandı.')

    processed_count, error_count = 0, 0

    for name, mini in tqdm(df.groupby('image_id'), desc='Veri seti oluşturuluyor'):
        if name in val_index:
            labels_dir = val_labels_dir
            images_dir = val_images_dir
        else:
            labels_dir = train_labels_dir
            images_dir = train_images_dir

        # label yaz
        label_file = os.path.join(labels_dir, f'{name}.txt')
        with open(label_file, 'w') as f:
            for _, row in mini.iterrows():
                class_id = int(row['classes'])
                x_center = row['x_center']
                y_center = row['y_center']
                width = row['w']
                height = row['h']
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        # görsel kopyala
        src_img = os.path.join(source_dir, f'{name}.jpg')
        dst_img = os.path.join(images_dir, f'{name}.jpg')
        if os.path.exists(src_img):
            sh.copy2(src_img, dst_img)
            processed_count += 1
        else:
            print('Görüntü bulunamadı:', src_img)
            error_count += 1

    print('\nVeri seti oluşturma tamamlandı!')
    print('Başarılı:', processed_count)
    print('Hatalı:', error_count)
    print('Çıktı klasörü:', output_dir)


In [ ]:
# Veri setini oluştur
create_yolo_dataset(df, index)


## Model Konfigürasyonu (Kısa ve Öz)
Eğitimde kullanacağım basit bir `yolo_config.yaml` dosyası üretiyorum. Burada tek sınıf olduğunu belirtiyorum ve başlangıç için makul hiperparametreler yazıyorum. Eğitim framework’üne göre bu dosyayı rahatça uyarlayabilirim.


In [ ]:
def create_model_config():
    config_content = """# YOLO Model Konfigürasyonu - Araç Tespiti
# Bu dosya YOLO model eğitimi için gerekli ayarları içerir

# Model parametreleri
nc: 1  # Sınıf sayısı (araç)
names: ['car']  # Sınıf isimleri

# Eğitim parametreleri
epochs: 100
batch_size: 16
img_size: 640
learning_rate: 0.01

# Veri yolları
train: yolo_dataset/images/train
val: yolo_dataset/images/val

# Çıktı klasörü
output_dir: runs/train
"""
    with open('yolo_config.yaml', 'w', encoding='utf-8') as f:
        f.write(config_content)
    print('Konfigürasyon dosyası yazıldı: yolo_config.yaml')

create_model_config()


## Tahmin Sonuçlarına Göz Atma (Opsiyonel Ama Eğlenceli)
Eğer bir tahmin (inference) aldıysam, sonuç klasörüne gidip rastgele bir görseli ekrana çağırıyorum. Böylece modelin verdiği kutular “mantıklı mı?” onu çıplak gözle test etmiş oluyorum.


In [ ]:
def visualize_prediction_results(results_dir='runs/detect/exp'):
    print('\nTahmin sonuçları analiz ediliyor...')
    predicted_files = []
    if os.path.exists(results_dir):
        for dirpath, dirnames, filenames in os.walk(results_dir):
            image_files = [f for f in filenames if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            predicted_files.extend(image_files)

    if not predicted_files:
        print('Sonuç bulunamadı:', results_dir)
        print('Model henüz çalıştırılmamış olabilir.')
        return

    print('Toplam tahmin dosyası:', len(predicted_files))
    random_file = random.choice(predicted_files)
    result_path = os.path.join(results_dir, random_file)
    print('Gösterilen dosya:', random_file)
    try:
        display(Image(filename=result_path))
    except Exception as e:
        print('Görüntü yüklenirken hata:', str(e))
        print('Dosya yolu:', result_path)

visualize_prediction_results()


## Örnekler ve Küçük İstatistikler
Finale doğru yaklaşırken, birkaç görsel üzerinde kutuları çizdirip “etiketler doğru yere oturuyor mu?” ona bakıyorum. Sonra da veri setim hakkında kısaca özet çıkarıyorum: kaç görüntü var, her görüntüde ortalama kaç araç var gibi…


In [ ]:
def display_sample_cars(num_samples=6):
    print(f"\n{num_samples} adet rastgele araç görüntüsü gösteriliyor...")
    sample_images = random.sample(index, min(num_samples, len(index)))

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('🚗 Araç Tespiti - Örnek Görüntüler', fontsize=16, fontweight='bold')

    for i, img_id in enumerate(sample_images):
        row, col = i // 3, i % 3
        img_path = f'training_images/{img_id}.jpg'
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_with_boxes = img_rgb.copy()

            car_data = df[df['image_id'] == img_id]
            for _, car in car_data.iterrows():
                x_center = car['x_center'] * img_w
                y_center = car['y_center'] * img_h
                width = car['w'] * img_w
                height = car['h'] * img_h
                x1 = int(x_center - width/2)
                y1 = int(y_center - height/2)
                x2 = int(x_center + width/2)
                y2 = int(y_center + height/2)
                cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img_with_boxes, 'CAR', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            axes[row, col].imshow(img_with_boxes)
            axes[row, col].set_title(f'ID: {img_id}\nAraç Sayısı: {len(car_data)}', fontsize=10, fontweight='bold')
            axes[row, col].axis('off')
        else:
            axes[row, col].text(0.5, 0.5, f'Görüntü bulunamadı\n{img_id}', ha='center', va='center', fontsize=10)
            axes[row, col].set_title(f'ID: {img_id}', fontsize=10)
            axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()
    print(f"{len(sample_images)} adet araç görüntüsü gösterildi!")


def show_car_statistics():
    print('\nARAÇ TESPİTİ İSTATİSTİKLERİ')
    print('=' * 50)
    total_cars = len(df)
    total_images = len(index)
    cars_per_image = total_cars / total_images

    print('Toplam araç sayısı:', total_cars)
    print('Toplam görüntü sayısı:', total_images)
    print('Görüntü başına ortalama araç:', f'{cars_per_image:.2f}')
    print('\nAraç Boyutları:')
    print('  - Ortalama genişlik (norm):', f"{df['w'].mean():.3f}")
    print('  - Ortalama yükseklik (norm):', f"{df['h'].mean():.3f}")
    print('  - En büyük araç:', f"{df['w'].max():.3f} x {df['h'].max():.3f}")
    print('  - En küçük araç:', f"{df['w'].min():.3f} x {df['h'].min():.3f}")

    cars_per_img = df.groupby('image_id').size().sort_values(ascending=False)
    print('\nEn çok araç içeren görüntüler:')
    for i, (img_id, count) in enumerate(cars_per_img.head(5).items(), start=1):
        print(f'{i}. {img_id}: {count} araç')


display_sample_cars(6)
show_car_statistics()
